In [1]:
# Tecnología
import pandas as pd
import os

# Conversiones

Hay muchos valores pérdidos en la métrica conversiones. Y la tabla que tiene conversiones su tipo es objeto. Por tanto, descartar este objetivo.

Cuando se tenga un número asociado a esta métrica, una posible solución es separar un dataframe que contiene conversiones y otro que contiene impresiones y reach. Serán seleccionados en función de la selección que haga el usuario en el front

In [2]:
# Get files path and names
path_directory = 'data/consolidate/'
list_path_files = os.listdir(path_directory)

# Concate all files in one dataframe
df = pd.DataFrame()
for file in list_path_files:
    path_file = path_directory + '/' + file
    df_iter = pd.read_csv(path_file)
    nodo = os.path.splitext(file)[0].split('_')[2]
    df_iter['nodos'] = nodo
    df = pd.concat([df, df_iter])

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 432 entries, 0 to 203
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   432 non-null    object 
 1   s3_key               432 non-null    object 
 2   permalink_url        432 non-null    object 
 3   creative_id          432 non-null    int64  
 4   adset_id             432 non-null    int64  
 5   campaign             432 non-null    object 
 6   campaign_id          432 non-null    int64  
 7   creative             432 non-null    object 
 8   created_time         432 non-null    object 
 9   name                 432 non-null    object 
 10  source_ad            155 non-null    object 
 11  id.1                 432 non-null    int64  
 12  partition_0          432 non-null    object 
 13  campaign_id.1        432 non-null    int64  
 14  campaign_name        432 non-null    object 
 15  impressions          432 non-null    int64  


In [3]:
# Agregar costo por reach
df['cpreach'] = df.spend.div(df.reach)
col_sel = ['spend', 'reach', 'impressions', 'clicks', 'cpreach', 'cpm', 'cpc']
df[col_sel]

,spend,reach,impressions,clicks,cpreach,cpm,cpc
0,939.45,46592,567720,2597,0.020163,1.654777,0.361744
1,401.50,11128,202201,1018,0.036080,1.985648,0.394401
2,335.50,9516,175237,697,0.035256,1.914550,0.481349
3,411.20,33888,251084,1204,0.012134,1.637699,0.341528
4,74.33,13184,43179,465,0.005638,1.721439,0.159849
...,...,...,...,...,...,...,...
199,0.02,77,81,0,0.000260,0.246914,NaN
200,1105.10,335805,996983,17950,0.003291,1.108444,0.061565
201,658.35,899839,1072552,1384,0.000732,0.613816,0.475686
202,2059.54,657025,1864523,44043,0.003135,1.104594,0.046762


In [8]:
# Agregar distancia entre dos puntos
def distancia_entre_2_puntos(x2, y2, x1, y1):
    distancia = (((x2 - x1)**2) + ((y2 - y1)**2))**(1/2)
    return distancia

In [21]:
# Crear distancias

# distancia awareness
df['dist_awar'] = distancia_entre_2_puntos(0, df.impressions.max(), df.cpm, df.impressions)

# distancia reach
df['dist_reach'] = distancia_entre_2_puntos(0, df.reach.max(), df.cpreach, df.reach)

# distancia consideración, traffic, clicks
df['dist_cons'] = distancia_entre_2_puntos(0, df.clicks.max(), df.cpc, df.clicks)

In [45]:
df[['dist_awar', 'dist_reach', 'dist_cons']]

,dist_awar,dist_reach,dist_cons
0,24374068.0,8.927733e+06,479814.000000
1,24739587.0,8.963197e+06,481393.000000
2,24766551.0,8.964809e+06,481714.000000
3,24690704.0,8.940437e+06,481207.000000
4,24898609.0,8.961141e+06,481946.000000
...,...,...,...
199,24941707.0,8.974248e+06,NaN
200,23944805.0,8.638520e+06,464461.000000
201,23869236.0,8.074486e+06,481027.000000
202,23077265.0,8.317300e+06,438368.000000


In [48]:
df[['dist_awar', 'dist_reach', 'dist_cons', 'clicks', 'cpc', 'spend']][df.dist_cons.isnull()]

,dist_awar,dist_reach,dist_cons,clicks,cpc,spend
214,2.494138e+07,8974026.0,NaN,0,NaN,0.34
39,2.488730e+07,8933741.0,NaN,0,NaN,451.09
40,2.490475e+07,8939949.0,NaN,0,NaN,316.15
113,2.494169e+07,8974232.0,NaN,0,NaN,0.05
115,2.494177e+07,8974307.0,NaN,0,NaN,0.00
199,2.494171e+07,8974248.0,NaN,0,NaN,0.02


In [49]:
df[['dist_awar', 'dist_reach', 'dist_cons', 'clicks', 'cpc', 'spend']]

,dist_awar,dist_reach,dist_cons,clicks,cpc,spend
0,24374068.0,8.927733e+06,479814.000000,2597,0.361744,939.45
1,24739587.0,8.963197e+06,481393.000000,1018,0.394401,401.50
2,24766551.0,8.964809e+06,481714.000000,697,0.481349,335.50
3,24690704.0,8.940437e+06,481207.000000,1204,0.341528,411.20
4,24898609.0,8.961141e+06,481946.000000,465,0.159849,74.33
...,...,...,...,...,...,...
199,24941707.0,8.974248e+06,NaN,0,NaN,0.02
200,23944805.0,8.638520e+06,464461.000000,17950,0.061565,1105.10
201,23869236.0,8.074486e+06,481027.000000,1384,0.475686,658.35
202,23077265.0,8.317300e+06,438368.000000,44043,0.046762,2059.54


In [23]:
import plotly.express as px
fig = px.scatter(df, y='reach', x='cpreach')
fig.show()

In [18]:
# Generate dataframe for conversiones, use df for awareness and consideración
df_conv = df[~df.conversions.isna()]
df_conv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 219 entries, 0 to 227
Data columns (total 36 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   219 non-null    object 
 1   s3_key               219 non-null    object 
 2   permalink_url        219 non-null    object 
 3   creative_id          219 non-null    int64  
 4   adset_id             219 non-null    int64  
 5   campaign             219 non-null    object 
 6   campaign_id          219 non-null    int64  
 7   creative             219 non-null    object 
 8   created_time         219 non-null    object 
 9   name                 219 non-null    object 
 10  source_ad            79 non-null     object 
 11  id.1                 219 non-null    int64  
 12  partition_0          219 non-null    object 
 13  campaign_id.1        219 non-null    int64  
 14  campaign_name        219 non-null    object 
 15  impressions          219 non-null    int64  


# Pregunta 1

¿Cuál es el significados de los objetivos? ¿Se agruparán todos en las categorías REACH, AWARENESS, CONSIDERACION, CONVERSION?

In [24]:
df['objective'].unique()

array(['OUTCOME_SALES', 'CONVERSIONS', 'LINK_CLICKS',
       'OUTCOME_ENGAGEMENT', 'OUTCOME_AWARENESS', 'PAGE_LIKES', 'REACH',
       'POST_ENGAGEMENT'], dtype=object)

In [25]:
dict_objetivos = {'AWARENESS':['OUTCOME_AWARENESS'],
                  'REACH': ['REACH'],
                  'CONSIDERACION':['LINK_CLICKS', 'OUTCOME_ENGAGEMENT', 'PAGE_LIKES', 'POST_ENGAGEMENT' ],
                  'CONVERSION':['CONVERSIONS', 'OUTCOME_SALES']}

In [26]:
df['nodos'].unique()

array(['plink', 'seguidores'], dtype=object)

In [76]:
def seleccionar_imagenes(dfp, dict_objetivos, nodo, objetivo, nom_col_nodo, nom_col_obj, met_awar, met_reach, met_cons, cant_imagenes = 4):
    # Printing
    print('')
    print('Diccionario con objetivos y sus posibles valores: ', dict_objetivos)
    print('')
    print('Nodo: ', nodo)
    print('Objetivo: ', objetivo)
    print('Nombre columna nodo: ', nom_col_nodo)
    print('Nombre columna objetivo: ', nom_col_obj)
    print('Metrica awareness: ', met_awar)
    print('Metrica reach: ', met_reach)
    print('Metrica consideración: ', met_cons)
    print('Cantidad de imágenes: ', cant_imagenes)

    # Selección Nodo
    dfp = dfp.loc[(dfp[nom_col_nodo] == nodo)]

    # AWARENESS
    if pd.Series(objetivo).isin(dict_objetivos['AWARENESS']).values[0]:
        dfp = dfp[dfp[nom_col_obj].str.contains(objetivo)]
        dfp = dfp.sort_values(met_awar, ascending = True)
        dfp = dfp[0:cant_imagenes]
        return dfp
    
    # REACH
    elif pd.Series(objetivo).isin(dict_objetivos['REACH']).values[0]:
        dfp = dfp[dfp[nom_col_obj].str.contains(objetivo)]
        dfp = dfp.sort_values(met_reach, ascending = True)
        dfp = dfp[0:cant_imagenes]
        return dfp
    
    # CONSIDERACION, TRAFFIC, CLICKS
    elif pd.Series(objetivo).isin(dict_objetivos['CONSIDERACION']).values[0]:
        dfp = dfp[dfp[nom_col_obj].str.contains(objetivo)]
        dfp = dfp.sort_values(met_cons, ascending = True)
        dfp = dfp[0:cant_imagenes]
        return dfp
    

In [81]:
# Valores

nodo = 'seguidores'
objetivo = 'REACH'
nombre_columna_nodo = 'nodos'
nombre_columna_objetivo = 'objective'
nombre_columna_imagenes = ''
nombre_columna_descripcion_imagen = ''
metrica_awar = 'dist_awar'
metrica_reach = 'dist_reach'
metrica_cons = 'dist_cons'
cantidad_imagenes = 6
seleccionar_imagenes(df, dict_objetivos, nodo, objetivo, nombre_columna_nodo, nombre_columna_objetivo, 
                     metrica_awar, metrica_reach, metrica_cons, cantidad_imagenes)


Diccionario con objetivos y sus posibles valores:  {'AWARENESS': ['OUTCOME_AWARENESS'], 'REACH': ['REACH'], 'CONSIDERACION': ['LINK_CLICKS', 'OUTCOME_ENGAGEMENT', 'PAGE_LIKES', 'POST_ENGAGEMENT'], 'CONVERSION': ['CONVERSIONS', 'OUTCOME_SALES']}

Nodo:  seguidores
Objetivo:  REACH
Nombre columna nodo:  nodos
Nombre columna objetivo:  objective
Metrica awareness:  dist_awar
Metrica reach:  dist_reach
Metrica consideración:  dist_cons
Cantidad de imágenes:  6


,id,s3_key,permalink_url,creative_id,adset_id,campaign,campaign_id,creative,created_time,name,...,actions,conversions,cost_per_conversion,action_values,partition_0.1,nodos,cpreach,dist_awar,dist_reach,dist_cons
203,3070598839871422:30bbd467aef7e70c8185721c75fc5002,images/act_3070598839871422/3070598839871422:3...,https://www.facebook.com/ads/image/?d=AQJ5Gmdf...,23849974853300192,23849974836690192,{id=23849974836540192},23849974836540192,{id=23849974853300192},2022-04-08T15:05:50-0500,DETRAS-DE-MUJER_006_POST,...,"[{action_type=onsite_conversion.post_save, val...",NaN,NaN,NaN,act_3070598839871422,seguidores,0.000655,9792719.0,6.554822e-04,474160.000001
171,3070598839871422:62429ad506442ba1cc00243774b1be6d,images/act_3070598839871422/3070598839871422:6...,https://www.facebook.com/ads/image/?d=AQKtaOk5...,23850874311100192,23850873800190192,{id=23850873800170192},23850873800170192,{id=23850874311100192},2022-07-27T10:38:06-0500,REEL 2 - ‘’¿No me lo dejas más barato?’’,...,"[{action_type=onsite_conversion.post_save, val...",NaN,NaN,NaN,act_3070598839871422,seguidores,0.000266,23075139.0,7.120372e+06,480785.000000
157,3070598839871422:24d2106d2e8b2ff5bf744c63bda1450b,images/act_3070598839871422/3070598839871422:2...,https://www.facebook.com/ads/image/?d=AQKTXLYD...,23851039102620192,23851039058710192,{id=23851039058760192},23851039058760192,{id=23851039102620192},2022-08-16T14:11:32-0500,REEL 4 - Soy Fira,...,"[{action_type=post, value=34}, {action_type=pa...",NaN,NaN,NaN,act_3070598839871422,seguidores,0.000299,22897992.0,7.353329e+06,480679.000000
159,3070598839871422:6dd60fcfa1048c75544dbe9fc9be52f5,images/act_3070598839871422/3070598839871422:6...,https://www.facebook.com/ads/image/?d=AQKR38WI...,23851011279090192,23851011228750192,{id=23851011228720192},23851011228720192,{id=23851011279090192},2022-08-12T17:13:33-0500,REEL 3 - Todo a su ritmo,...,"[{action_type=onsite_conversion.post_save, val...",NaN,NaN,NaN,act_3070598839871422,seguidores,0.000200,24033543.0,8.066080e+06,481676.000000
183,3070598839871422:6ebd1dd44ea2faa26c5a2f94989ffef4,images/act_3070598839871422/3070598839871422:6...,https://www.facebook.com/ads/image/?d=AQKX8X_4...,23850806359020192,23850806333640192,{id=23850806333620192},23850806333620192,{id=23850806359020192},2022-07-19T11:50:09-0500,Reels 2,...,"[{action_type=onsite_conversion.post_save, val...",NaN,NaN,NaN,act_3070598839871422,seguidores,0.000173,24264455.0,8.317818e+06,480910.000000
170,3070598839871422:06ab794b9775b359112ccaeb72ae089f,images/act_3070598839871422/3070598839871422:0...,https://www.facebook.com/ads/image/?d=AQJKYWad...,23850874311270192,23850873800190192,{id=23850873800170192},23850873800170192,{id=23850874311270192},2022-07-27T10:38:06-0500,REEL 4 - El que quiere puede y pa`cuando,...,"[{action_type=post, value=40}, {action_type=pa...",NaN,NaN,NaN,act_3070598839871422,seguidores,0.000261,24348331.0,8.380868e+06,481718.000000
